In [10]:
import pandas as pd
import pylab as p
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import ShuffleSplit, train_test_split
import scipy
import random
import time
from multiprocessing import Pool

In [2]:
core_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/CORE.csv')
game_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/GAMES.csv', index_col=[0])
#win_orig_df = win_orig_df.join(game_df)
offense_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/OFFENSE.csv', index_col=[0])
defense_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/DEFENSE.csv', index_col=[0])
players_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/PLAYERS.csv', index_col=[0])
# players_df['player_id'] = players_df.index
# offense_df = offense_df.join(players_df)
# defense_df = defense_df.join(players_df)


In [3]:
print offense_df.columns

Index([u'GID', u'PLAYER', u'PA', u'PC', u'PY', u'INT', u'TDP', u'RA', u'SRA', u'RY', u'TDR', u'TRG', u'REC', u'RECY', u'TDRE', u'FUML', u'PENY', u'FPTS', u'GAME', u'SEAS', u'YEAR', u'TEAM'], dtype='object')


In [4]:
teams = offense_df.groupby('TEAM')
for key, val in teams:
    print key

ARI
ATL
BAL
BUF
CAR
CHI
CIN
CLE
DAL
DEN
DET
GB
HOU
IND
JAC
KC
MIA
MIN
NE
NO
NYG
NYJ
OAK
PHI
PIT
SD
SEA
SF
STL
TB
TEN
WAS


In [21]:
ignore_columns = ['PLAYER', 'FPTS', 'GID']
features = []
total_count = len(players_df.index)
for j, player_id in enumerate(players_df.index):
    offense_player_plays = offense_df[offense_df['PLAYER'] == player_id]
#     defense_player_plays = defense_df[defense_df['PLAYER'] == player_id]
    if j%1000 == 0:
        print "{0}/{1}".format(j, total_count)
    if len(offense_player_plays) > 0:
#         print len(offense_player_plays)
        for i,row in enumerate(offense_player_plays.iterrows()):
            # target is this games fantasy points
            feature = {'target':row[1]['FPTS']}
            # features are all the older games not including this one [i:] historical data
            # this means there is a feature for every game for each player
            for k, row in enumerate(offense_player_plays[i+1:(i+20)].iterrows()):
                for key, val in row[1].iteritems():
                    if key in ignore_columns:
                        continue
                    if key == "TEAM":
                        feature["IS_{0}_{1}".format(val, k)] = 1.0
                    else:
                        feature["{0}_{1}".format(key, k)] = val
            features.append(feature)
#         if j > 10:
#             break
        

0/6541
1000/6541
2000/6541
3000/6541
4000/6541
5000/6541
6000/6541


In [22]:
features_df = pd.DataFrame(features)
features_df = features_df.fillna(0.0)
print features_df

       FUML_0  FUML_1  FUML_10  FUML_11  FUML_12  FUML_13  FUML_14  FUML_15  \
0           0       0        0        0        0        0        0        0   
1           0       0        0        0        0        0        0        0   
2           0       0        0        0        0        0        0        0   
3           0       0        0        0        0        0        0        0   
4           0       0        0        0        0        0        0        0   
5           0       0        0        0        0        0        0        0   
6           0       0        0        0        0        0        0        0   
7           0       0        0        0        0        0        0        0   
8           0       0        0        0        0        0        0        0   
9           0       0        0        0        0        0        0        0   
10          0       0        0        0        0        0        0        0   
11          0       0        0        0        0    

In [23]:
features_df = features_df.fillna(0.0)
if 'target' in features_df.columns:
    targets = features_df['target']
if 'target' in features_df.columns:
    del features_df['target']
x_train, x_test, y_train, y_test = train_test_split(features_df, targets, test_size=0.33, random_state=42)
cfr = ExtraTreesRegressor(
    oob_score = True, bootstrap=True, verbose = 100, max_features = None, n_estimators = 100, min_samples_leaf = 1,
    n_jobs=7,
    random_state=0,
)
cfr.fit(x_train, y_train)



building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 7 of 100
building tree 6 of 100
building tree 8 of 100[Parallel(n_jobs=7)]: Done   2 out of 100 | elapsed:   20.3s remaining: 16.6min

building tree 9 of 100
[Parallel(n_jobs=7)]: Done   6 out of 100 | elapsed:   21.0s remaining:  5.5min
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
[Parallel(n_jobs=7)]: Done   4 out of 100 | elapsed:   22.1s remaining:  8.8min
building tree 14 of 100
[Parallel(n_jobs=7)]: Done   1 out of 100 | elapsed:   22.7s remaining: 37.5min
building tree 15 of 100
[Parallel(n_jobs=7)]: Done   8 out of 100 | elapsed:   42.5s remaining:  8.1min
building tree 16 of 100
[Parallel(n_jobs=7)]: Done  10 out of 100 | elapsed:   42.6s remaining:  6.4min
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
[Parallel(n_jobs=7)]: Done  14 out of 100 | elaps

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=100, n_jobs=7, oob_score=True, random_state=0,
          verbose=100, warm_start=False)

In [24]:
score = cfr.score(x_test, y_test)
print score

[Parallel(n_jobs=7)]: Done   2 out of 100 | elapsed:    0.0s remaining:    0.9s
[Parallel(n_jobs=7)]: Done   1 out of 100 | elapsed:    0.0s remaining:    1.8s
[Parallel(n_jobs=7)]: Done   4 out of 100 | elapsed:    0.0s remaining:    0.6s
[Parallel(n_jobs=7)]: Done   6 out of 100 | elapsed:    0.0s remaining:    0.5s
[Parallel(n_jobs=7)]: Done   8 out of 100 | elapsed:    0.0s remaining:    0.4s
[Parallel(n_jobs=7)]: Done  10 out of 100 | elapsed:    0.0s remaining:    0.4s
[Parallel(n_jobs=7)]: Done  12 out of 100 | elapsed:    0.1s remaining:    0.4s
[Parallel(n_jobs=7)]: Done  14 out of 100 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=7)]: Done  16 out of 100 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=7)]: Done  20 out of 100 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=7)]: Done  18 out of 100 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=7)]: Done  22 out of 100 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=7)]: Done  24 out of 10